In [8]:
import numpy as np 
import pandas as pd 
import os
import csv
import warnings
warnings.simplefilter(action='ignore')
import gc
from pathlib import Path 


In [9]:
df_combined = pd.read_csv('sampled_dataset_full_column.csv',lineterminator='\n')

print(len(df_combined))

columns = ["userid","username","location","tweetid","tweetcreatedts","retweetcount", "hashtags", "text", "usercreatedts", "following", "followers","totaltweets"]

# select equal timeframe bafore and after the ban of twitter in Russia
df1 = df_combined[columns][df_combined['tweetcreatedts'] > "2022-02-20 00:00:00" ] 
df1 = df1[df1['tweetcreatedts'] < "2022-03-14 00:00:00" ] 
df1.shape

163204


(53158, 12)

In [10]:
df1.sample(3)

,userid,username,location,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets
67033,2971291040,syakirharis25,"Planet Earth, Solar System, 🌌",1497764327528353794,2022-02-27 02:43:37,4,[],"In Lviv, at the station, people are waiting fo...",2015-01-10 06:56:04.000000,4,405,25295
90981,1458160062594097159,IAminnocentYES1,NaN,1500896941860810752,2022-03-07 18:11:30,271,"[{'text': 'Ukraine', 'indices': [114, 122]}]",A German multi-billionaire has rented three ho...,2021-11-09 19:52:02.000000,438,175,25531
97951,2323385174,Fandango831,NaN,1501580989289619457,2022-03-09 15:29:40,209,"[{'text': 'Ukraine', 'indices': [15, 23]}, {'t...","#Ukraine: Apparently, the Territorial Defense ...",2014-02-02 07:34:07.000000,134,52,2444


### Hashtags

### (1) top used hashtags 

In [11]:
tags = df1['hashtags'].tolist()

TAGS=[]
for item in tags:
    TAGS+=item[1:-1].split('{')
TAGS2=[]
for item2 in TAGS:
    if item2!='':
        TAGS2+=[item2.split(': ')[1].split(',')[0][1:-1]]
tagdf=pd.DataFrame(pd.Series(TAGS2).value_counts())
tagdf['tag']=tagdf.index.tolist()
tagdf.columns=['counts','tag']
tagdf.index=range(len(tagdf))

tag_c = tagdf[tagdf['counts']>30]
tag_c.to_csv("hashtag_count_over30.csv",sep = ",", header = True, encoding = "UTF-8",index=False)

In [12]:
tag_c.sample(6)

,counts,tag
151,59,BreakingNews
123,72,RUSSIA
59,193,kyiv
245,31,Hitler
86,113,StopTheWar
120,75,IWD2022


### (2) clean hashtags for each tweet 

In [13]:
# write a function to extract "hashtags" 
def SplitTags(hashtags):
    TAGS=[]
    if hashtags != '[]':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]
    if len(TAGS)>0:
        return TAGS
    else: return "NA"

def LowerTags(hashtags):
    TAGS=[]
    if hashtags != '[]':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]
    lower_TAGS = (map(lambda x: x.lower(), TAGS))
    TAGS = list(lower_TAGS)
    if len(TAGS)>0:
        return TAGS
    else: return "NA"

            
df1['tags_splitted'] = df1['hashtags'].apply(SplitTags)
df1['tags_lower'] = df1['hashtags'].apply(LowerTags)
df1.sample(3)

,userid,username,location,tweetid,tweetcreatedts,retweetcount,hashtags,text,usercreatedts,following,followers,totaltweets,tags_splitted,tags_lower
68783,1498237580999077892,lita_ob,NaN,1498274162862870539,2022-02-28 12:29:31,0,"[{'text': 'UKRAINE', 'indices': [43, 51]}, {'t...",@ediramaal STOP RUSSIAN AGGRESSION AGAINST #UK...,2022-02-28 10:04:54.000000,2,0,24,"[UKRAINE, StopPutin, StopRussia]","[ukraine, stopputin, stoprussia]"
104217,1447248997253406723,Nefarious_Actz_,"Los Angeles, CA",1502372997281157123,2022-03-11 19:56:49,5,"[{'text': 'Ukrainian', 'indices': [36, 46]}, {...",Z marks the spot. #Ukrainian forces defending ...,2021-10-10 17:13:56.000000,599,39,6221,"[Ukrainian, Mariupol]","[ukrainian, mariupol]"
107775,1494227064408346634,Deramba1,NaN,1502777501138923521,2022-03-12 22:44:11,25,"[{'text': 'Ukraine', 'indices': [57, 65]}, {'t...",While the world continues to focus on #Ukraine...,2022-02-17 08:29:07.000000,270,276,4332,"[Ukraine, Tigray]","[ukraine, tigray]"


### (3) create node attribute of political orientation

In [14]:
# import hashtag file 
df_tags = pd.read_csv("hashtag_over30_tagged.csv")

pro_R_tags = df_tags[df_tags["Pro_R"]=="TRUE"]['tag'].to_list()

pd.crosstab(index=df_tags['Pro_R'], columns='count') # 5
pd.crosstab(index=df_tags['Pro_U'], columns='count') #61
pd.crosstab(index=df_tags['Neutral'], columns='count') #178

df_tags.sample(5)

,counts,tag,Pro_U,Pro_R,Neutral
139,40,Zelenskiy,False,FALSE,True
136,42,BTC,False,FALSE,True
156,35,Czech,False,FALSE,True
144,39,Nazi,False,FALSE,True
89,64,Iraq,False,FALSE,True


In [15]:
pro_R_tags = ["IStandWithPutin","istandwithrussia","ISupportRussia","ZelenskyWarCriminal","Zenlenskyisawarcriminal",\
              "NaziUkraine","UkraineNazis","nazisinukraine","WarCrimesOfUkraine","PutinIsRight","StopHatingRussians",\
              "NATOWarCriminal", "PatrioticRussia"]
pro_U_tags = ["Istandwithukraine","Putinisawarcriminal","PutinWarCriminal","StandWithUkraine", "HandsOffUkraine",\
              "kievfree","UkraineUnderAttack","StopRussia","StopPutin","RussiaInvadedUkriane","StopPutinsWar",\
              "StopRussianAggression","ukraineinvasion","russianinvasion","RussiaInvadedUkraine","PrayingForUkraine",\
              "RussiaOutOfUkraine","StandWithUkriane","RussiaInvadesUkraine","PutinHitler"]

# change all tags to lowercase before matching to avoid spelling difference
lower_R = (map(lambda x: x.lower(), pro_R_tags))
pro_R_tags = list(lower_R)

lower_U = (map(lambda x: x.lower(), pro_U_tags))
pro_U_tags = list(lower_U)

# lambda functions marking political orientation if related hashtags were found 
pro_R_check = lambda taglist: bool(set(taglist) & set(pro_R_tags)) # return true if overlap of two sets not empty
pro_U_check = lambda taglist: bool(set(taglist) & set(pro_U_tags)) 

df1['pro_Russian'] = df1['tags_lower'].apply(pro_R_check)
df1['pro_Ukraine'] = df1['tags_lower'].apply(pro_U_check)

In [16]:
df1.shape

(53158, 16)

In [17]:
no_tag = df1[df1['tags_splitted']=="NA"]
# 10594 tweets with no hashtags

# 255 labeled as pro-Russia
proRussia = df1[df1['pro_Russian']==True]
proRussiaNoUkraine = df1[(df1['pro_Russian']==True) & (df1['pro_Ukraine']==False)]

# 6412 labeled as pro-Ukraine
proUkraine = df1[df1['pro_Ukraine']==True]
proUkraineNoRussia = df1[(df1['pro_Russian']==False) & (df1['pro_Ukraine']==True)]

# still a lot rabeled by neither of two groups
non_R = df1[df1['pro_Russian']==False]
# non = non_R[non_R['pro_Ukraine']==False]

non = df1[(df1['pro_Russian']==False) & (df1['pro_Ukraine']==False)]
both = df1[(df1['pro_Russian']==True) & (df1['pro_Ukraine']==True)]

display(proRussia.shape)
display(proUkraine.shape)
display(non.shape)
display(both.shape)
display(proRussiaNoUkraine.shape)
display(proUkraineNoRussia.shape)


(255, 16)

(6412, 16)

(46508, 16)

(17, 16)

(238, 16)

(6395, 16)

In [18]:
both["text"].to_list()

['What I was thinking, did you too? \n#UkraineRussiaWar #UkraineUnderAttack #IStandWithPutin #UkraineKrieg #RussianUkrainianWar https://t.co/jGwpoZc42J',
 '@GazetaRu #IStandWithPutin going to #TheHague then the #Gulag   #EndPutin #DeposePutin #StopPutin #OustPutin #RussianRevolution https://t.co/XScqkZpwWH',
 '@BlackSoul_AZ By choosing #istandwithputin you are choosing be responsible for killing children in #ukraine #shelteroursky #noflyzoneoverukraine \n\n #putin #standwithukraine #SlavaUkraini\n#StopWar #StopRussia #UkraineWillPrevail https://t.co/W5LNG0Hvan',
 'The Debacle facing the world 🌎!! Where do You Stand!? 🇷🇺🇺🇦#RussianUkrainianWar #Russia #Ukraine #istandwithrussia #IStandWithUkraine #Resolution #Zelenskyy #Putin #Kiyv #Kremlin https://t.co/Wu8DiBfJhJ',
 'If #Russia doesn’t fight and take out it’s dictatorship #Poutine then I can’t help but process how alot of Russian civilians are going to starve &amp; be hated around the world. #IStandWithUkraine #IStandWithPutin #istandwi

In [19]:
df_R = non[non['text'].str.contains("zelensky" and "greenscreen")]
# tried keywords “Zenlensky” AND “Greenscreen” but no results in the non-labeled
df_R.shape

(0, 16)

### Nodelist with political stance

In [29]:
node_columns = ["userid","username","usercreatedts","following","followers","totaltweets","pro_Russian","pro_Ukraine"]
nodesBefore = df1[node_columns][(df1['tweetcreatedts'] < '2022-03-04 24:59:59+00:00')]
nodesAfter = df1[node_columns][(df1['tweetcreatedts'] > '2022-03-04 24:59:59+00:00')]

difAll = round(len(nodesAfter) * 100 / len(nodesBefore), 2)
print("Tweets before Russia's block:", len(nodesBefore), "and after:", len(nodesAfter), "\nDifference:", difAll, len(nodesBefore)-len(nodesAfter))

proRussianBefore = proRussia[node_columns][(proRussia['tweetcreatedts'] < '2022-03-04 24:59:59+00:00')]
proRussianAfter = proRussia[node_columns][(proRussia['tweetcreatedts'] > '2022-03-04 24:59:59+00:00')]
difRus = round(len(proRussianAfter) * 100 / len(proRussianBefore),2)
print("Pro-Russian tweets before Russia's block:", len(proRussianBefore), "and after:", len(proRussianAfter), "\nDifference:", difRus, len(proRussianBefore)-len(proRussianAfter))

ratioBefore = round(len(proRussianBefore)/len(nodesBefore), 4)
ratioAfter = round(len(proRussianAfter)/len(nodesAfter), 4)

print("Ratio pro-Russian:Total tweets before block:", ratioBefore, "and after:", ratioAfter)


proUkraineBefore = proUkraine[node_columns][(proUkraine['tweetcreatedts'] < '2022-03-04 24:59:59+00:00')]
proUkraineAfter = proUkraine[node_columns][(proUkraine['tweetcreatedts'] > '2022-03-04 24:59:59+00:00')]
difUkr = round(len(proUkraineAfter) * 100 / len(proUkraineBefore),2)
print("Pro-Ukrainian tweets before Russia's block:", len(proUkraineBefore), "and after:", len(proUkraineAfter), "\nDifference:", difUkr, len(proUkraineBefore)-len(proUkraineAfter))


bothBefore = both[node_columns][(both['tweetcreatedts'] < '2022-03-04 24:59:59+00:00')]
bothAfter = both[node_columns][(both['tweetcreatedts'] > '2022-03-04 24:59:59+00:00')]
difBoth = round(len(bothAfter) * 100 / len(bothBefore),2)
print("Both tweets before Russia's block:", len(bothBefore), "and after:", len(bothAfter), "\nDifference:", difBoth, len(bothBefore)-len(bothAfter))


nonBefore = non[node_columns][(non['tweetcreatedts'] < '2022-03-04 24:59:59+00:00')]
nonAfter = non[node_columns][(non['tweetcreatedts'] > '2022-03-04 24:59:59+00:00')]
difNon = round(len(nonAfter) * 100 / len(nonBefore),2)
print("Neutral tweets before Russia's block:", len(nonBefore), "and after:", len(nonAfter), "\nDifference:", difNon, len(nonBefore)-len(nonAfter))

Tweets before Russia's block: 23156 and after: 30002 
Difference: 129.56 -6846
Pro-Russian tweets before Russia's block: 169 and after: 86 
Difference: 50.89 83
Ratio pro-Russian:Total tweets before block: 0.0073 and after: 0.0029
Pro-Ukrainian tweets before Russia's block: 2741 and after: 3671 
Difference: 133.93 -930
Both tweets before Russia's block: 7 and after: 10 
Difference: 142.86 -3
Neutral tweets before Russia's block: 20253 and after: 26255 
Difference: 129.64 -6002


In [21]:
print(23156+30002)

53158


In [22]:
# code for nodetype variable
def Poli_Stance(Pro_R, Pro_U):
    if df1[Pro_R] == "True":
        return "R"
    elif df1[Pro_U] == "True":
        return "U"
    else:
        return "NA"
    
#nodelist["NodeType"] = Poli_Stance(df1["pro_Russian"],df1["pro_Ukraine"])

# Determine a user type by it's hashtags: pro-russian / pro-ukrainian / unknown
nodesBefore['NodeType'] = np.where(
     nodesBefore['pro_Russian'] & nodesBefore['pro_Ukraine'], 'Both',
     np.where(
         nodesBefore['pro_Russian'],  'pro_Russian',
     np.where(
        nodesBefore['pro_Ukraine'], 'pro_Ukraine', 'Unknown'
     )))

nodesAfter['NodeType'] = np.where(
    nodesAfter['pro_Russian'] & nodesAfter['pro_Ukraine'], 'Both',
    np.where(
    nodesAfter['pro_Russian'], 'pro_Russian', 
    np.where(
        nodesAfter['pro_Ukraine'], 'pro_Ukraine', 'Unknown'
    )))

In [23]:
PUB = len(nodesBefore[nodesBefore["NodeType"] == "pro_Ukraine"])
PUA = len(nodesAfter[nodesAfter["NodeType"] == "pro_Ukraine"])
PRB = len(nodesBefore[nodesBefore["NodeType"] == "pro_Russian"])
PRA = len(nodesAfter[nodesAfter["NodeType"] == "pro_Russian"])
print(PRB+PRA)
# Need to eliminate

238


In [24]:
#pd.crosstab(index=nodelist['pro_Russian'], columns='count')
x = pd.crosstab(index=proRussianBefore['username'], columns='count')
print(x[x["count"] > 1])
x = pd.crosstab(index=proRussianAfter['username'], columns='count')
print(x[x["count"] > 1])

col_0      count
username        
tindian88      2
col_0            count
username              
Free_Libyan2022      3
KellyY67122045       2
Real_Jayant          2
aficionali           2


In [25]:
# del
## merge with node in the retweet projected graph
# userNodes = pd.read_csv("user_nodes.csv")
# graph1_nodes = set(userNodes["Source"])
# nodes1 = set(nodelist1["userid"])
# overlap1 = list(graph1_nodes & nodes1)

# overlap1

projected_w_user_edges_2 = pd.read_csv("projected_w_user_edgelist_2.csv")
graph2_nodes = set(projected_w_user_edges_2["Target"])
nodes2 = set(nodesAfter["userid"].apply(lambda x: int(x)))
overlap2 = list(graph2_nodes & nodes2)
overlap2

# graph1_nodes

[1222614577860444160,
 1235907262771757057,
 1028450599157858304,
 1306625045758869507,
 1304888715412635652,
 758229561465339904,
 1222080151946657798,
 1267224277511700487,
 801877950819958784,
 1272858471663370240,
 1111645962240491523,
 1501116361250308099,
 1334335235719266307,
 1367192569600376836,
 1441917272952504325,
 1390710257534468102,
 1497584806036602888,
 1049345996990414856,
 1372920730451607561,
 1490813866703470597,
 1478824607599706117,
 2345631768,
 382763115,
 1321717900143693826,
 796488837996351488,
 758275784188497920,
 727698631705366528,
 760651402741944320,
 704667424646369280,
 784620251849822208,
 1326764182239318016,
 339017860,
 1383017797379297281,
 1417682185394458625,
 1448117527712763905,
 819398345739341825,
 749340736055808001,
 1373108174836862977,
 830448180915474433,
 1446165808246296579,
 1330723807561125890,
 1330003168030232578,
 1398394923821436931,
 1446570297528815621,
 1370075261425815559,
 1500878880470667268,
 1392564395377627140,
 13689

In [26]:
nodelist1.to_csv("nodelist1_poli.csv",sep = ",", header = True, encoding = "UTF-8",index=False)
nodelist2.to_csv("nodelist2_poli.csv",sep = ",", header = True, encoding = "UTF-8",index=False)

NameError: name 'nodelist1' is not defined

### Edgelist for user->hashtag mapping

In [ ]:
hashtags = df1['tags_splitted'][0]


['Ukraine', 'UkraineRussia', 'Putin', 'UkraineInvasion', 'Ukrainian', 'แตงโม']

In [ ]:
def UHmapping(hashtags):
    TAGS=[]
    if hashtags != 'NA':
        tags_split = hashtags[1:-1].split('{')
        for t in tags_split:
            if t!='':
                TAGS+=[t.split(': ')[1].split(',')[0][1:-1]]

edgelist['hashtag'] = df1['hashtags'].apply(UHmapping)
                          

                          

In [ ]:
edgelist = {'username':[],'tag':[]}
#df1 = df1.reset_index()  # make sure indexes pair with number of rows
for index, row in df1.iterrows():
    user = row['username']
    hashtags = row['tags_splitted']
    # change pandas series to list
    if hashtags != "NA":
        hashtags = list(pd.Series(hashtags).values)
        for tag in hashtags:
            edgelist.setdefault('username',[]).append(user)
            edgelist.setdefault('tag',[]).append(tag) 

edgelist = pd.DataFrame.from_dict(edgelist)

In [ ]:
edgelist

,username,tag
0,felionaaaa,Ukraine
1,felionaaaa,UkraineRussia
2,felionaaaa,Putin
3,felionaaaa,UkraineInvasion
4,felionaaaa,Ukrainian
...,...,...
108769,bigwincam,UKRAINE
108770,liontwee,Putin
108771,liontwee,StopPutinsWar
108772,liontwee,PutinIsaWarCriminal


In [ ]:
edgelist.to_csv("edgelist_user_tag.csv",sep = ",", header = True, encoding = "UTF-8",index=False)